## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP1 - Verificación de la calidad de los datos

---

En esta libreta se realiza una verificación de calidad de datos con base en los planteado en el Anexo SP3 y SP4. 

---

In [1]:
#Se importan las librerias a utilizar

import pandas as pd

----

##### Lectura de los datasets

**ANTES DE EJECUTAR**:
- Copiar los archivos de datos al directorio processed para modificarlos eventualmente ahí. En este caso se agrega al final "_c" para identificarlos.
- Versionarlos con dvc y hacer el paso correspondiente en git.
- Hacer la lectura directo desde esos nuevos archivos.

In [2]:
df_creditos = pd.read_csv("../../data/processed/datos_creditos_c.csv", sep=";")
display(df_creditos.head(1))

df_tarjetas = pd.read_csv("../../data/processed/datos_tarjetas_c.csv", sep=";")
display(df_tarjetas.head(1))

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,713061558.0,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,Blue,24.0,2.0


---
#### Verificación de calidad de datos

**Análisis a realizar**

1. Evaluación de valores nulos (filas y columnas)
2. Evaluación de formato válido
3. Valores ajustados en rangos (ver anexos)
4. Claves únicas
5. Integridad referencial
6. Cumplimiento de reglas en valores

In [3]:
# Establecimiento de los umbrales de aceptación

FORMATEO_VALORES = 0.1
RANGOS_VALORES = 0.0
INTEGRIDAD_REF = 0.1
REGLAS_VALORES = 0.1

# Valores globales

cantidad_filas_creditos = df_creditos.shape[0]
cantidad_filas_tarjetas = df_tarjetas.shape[0]

## Dimensión: completitud

### (1a) Filas

In [4]:
# Se obtienen las cantidades de valores nulos por columna

nulos_x_columna_c = df_creditos.isna().sum()
nulos_x_columna_t = df_tarjetas.isna().sum()

print(f"Cantidad de filas que tienen valores nulos por atributo:\n{nulos_x_columna_c}\n")
print(f"Cantidad de filas que tienen valores nulos por atributo:\n{nulos_x_columna_t}")

Cantidad de filas que tienen valores nulos por atributo:
id_cliente               0
edad                     0
importe_solicitado       0
duracion_credito         0
antiguedad_empleado    337
situacion_vivienda       0
ingresos                 0
objetivo_credito         0
pct_ingreso              0
tasa_interes           912
estado_credito           0
falta_pago               0
dtype: int64

Cantidad de filas que tienen valores nulos por atributo:
id_cliente             0
antiguedad_cliente     0
estado_civil           0
estado_cliente         0
gastos_ult_12m         0
genero                 0
limite_credito_tc      0
nivel_educativo        0
nivel_tarjeta          0
operaciones_ult_12m    0
personas_a_cargo       0
dtype: int64


In [5]:
# De cualquier manera se establece el cálculo a realizar

cantidad_columnas = len(df_creditos.axes[1])

df_creditos['completitud_fila'] = (df_creditos.isnull().sum(axis=1) / cantidad_columnas)

problemas = df_creditos[df_creditos['completitud_fila'] >= 0.2]

completitud_f = problemas.shape[0]

print("Filas que incumplen el umbral de nulos en columnas [completitud_f] - créditos - :")
print(f"{completitud_f} ({round((completitud_f  / cantidad_filas_creditos) * 100, 2)})%")

Filas que incumplen el umbral de nulos en columnas [completitud_f] - créditos - :
0 (0.0)%


In [6]:
# De cualquier manera se establece el cálculo a realizar

cantidad_columnas = len(df_tarjetas.axes[1])

df_tarjetas['completitud_fila'] = (df_tarjetas.isnull().sum(axis=1) / cantidad_columnas)

problemas = df_tarjetas[df_tarjetas['completitud_fila'] >= 0.2]

completitud_f = problemas.shape[0]

print("Filas que incumplen el umbral de nulos en columnas [completitud_f] - tarjetas - :")
print(f"{completitud_f} ({round((completitud_f  / cantidad_filas_creditos) * 100, 2)})%")

Filas que incumplen el umbral de nulos en columnas [completitud_f] - tarjetas - :
0 (0.0)%


### (1b) Dataset

In [7]:
completitud_dc = df_creditos.isnull().any(axis=1).sum()

print("Filas que presentan nulos en el dataset [completitud_d] - creditos - :")
print(f"{completitud_dc} ({round((completitud_dc  / cantidad_filas_creditos) * 100, 2)})%\n")

completitud_dt = df_tarjetas.isnull().any(axis=1).sum()

print("Filas que presentan nulos en el dataset [completitud_d] - tarjetas - :")
print(f"{completitud_dt} ({round((completitud_dt  / cantidad_filas_tarjetas) * 100, 2)})%")

Filas que presentan nulos en el dataset [completitud_d] - creditos - :
1225 (12.1)%

Filas que presentan nulos en el dataset [completitud_d] - tarjetas - :
0 (0.0)%


----

## Dimensión: exactitud

### (2) Formato válido

In [ ]:
# No se encuentran atributos con formato específico

----

## Dimensión: exactitud

### (3) Valores ajustados

Atributo: **edad**

In [8]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: edad

valores = df_creditos['edad'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['edad']) - df_creditos['edad'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['edad'] > 90]

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
edad
22     2228
23     2190
24     1926
25     1646
26     1328
21      799
20        6
144       2
123       2
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila
81,715623483.0,144,4800,3,4.0,ALQUILER,250000,INVERSIONES,0.02,13.57,0,N,0.000000
183,815500983.0,144,6000,2,4.0,HIPOTECA,200000,EDUCACIÓN,0.03,11.86,0,N,0.000000
575,717956283.0,123,20400,3,2.0,ALQUILER,80004,EDUCACIÓN,0.25,10.25,0,N,0.000000
747,712881483.0,123,20000,4,7.0,ALQUILER,78000,INVERSIONES,0.26,NaN,0,N,0.083333


Cantidad detectada: 4


In [9]:
def calcular_rangos_valores_edad():
    edad_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo edad: {edad_valores_fuera_rango}")
    
    indicador = (edad_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo edad): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_edad()

Cantidad de filas con valores fuera de rango en atributo edad: 4
Porcentaje de filas con errores de rango de valores (atributo edad): 0.04 %
Evaluación: no cumplimiento


In [10]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: importe

valores = df_creditos['importe_solicitado'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['importe_solicitado']) - df_creditos['importe_solicitado'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['importe_solicitado'] < 0]

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
importe_solicitado
5000     800
6000     715
8000     556
4000     446
7000     436
        ... 
29800      1
9325       1
9375       1
34800      1
9425       1
Name: count, Length: 509, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila


Cantidad detectada: 0


In [11]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: duracion_credito

valores = df_creditos['duracion_credito'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['duracion_credito']) - df_creditos['duracion_credito'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['duracion_credito'] < 0]

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
duracion_credito
2    3404
3    3364
4    3359
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila


Cantidad detectada: 0


In [12]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: antiguedad_empleado

valores = df_creditos['antiguedad_empleado'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['antiguedad_empleado']) - df_creditos['antiguedad_empleado'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[df_creditos['antiguedad_empleado'] > 50]

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
antiguedad_empleado
0.0      1414
2.0      1281
3.0      1128
5.0      1050
6.0      1027
1.0       993
4.0       846
7.0       784
8.0       587
9.0       414
10.0      237
11.0       27
123.0       2
Name: count, dtype: int64

Cantidad de nulos en el atributo: 337

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y,0.0
210,715999608.0,21,20000,4,123.0,HIPOTECA,192000,INVERSIONES,0.10,6.54,0,N,0.0


Cantidad detectada: 2


In [13]:
def calcular_rangos_valores_ae():
    ae_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo edad: {ae_valores_fuera_rango}")
    
    indicador = (ae_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo antiguedad_empleado): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_ae()

Cantidad de filas con valores fuera de rango en atributo edad: 339
Porcentaje de filas con errores de rango de valores (atributo antiguedad_empleado): 3.35 %
Evaluación: no cumplimiento


Se procesa el atributo: **situacion_vivienda**


In [14]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: situacion_vivienda

valores = df_creditos['situacion_vivienda'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['situacion_vivienda']) - df_creditos['situacion_vivienda'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida
    
valores_validos = 'ALQUILER|PROPIA|HIPOTECA|OTROS' # Se define una re de los valores validos según el anexo

df_creditos['situacion_vivienda_ok'] = df_creditos['situacion_vivienda'].astype(str).str.match(valores_validos)

print("Se visualizan las filas con errores de rango:")
display(df_creditos[~df_creditos['situacion_vivienda_ok']]) # Para visualizar las tuplas con valores nulos o erróneos

# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[~df_creditos['situacion_vivienda_ok']] 
print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
situacion_vivienda
ALQUILER    6125
HIPOTECA    3223
PROPIA       741
OTROS         38
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok


Cantidad detectada: 0


In [15]:
def calcular_rangos_valores_situacion_vivienda():
    situacion_vivienda_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo situacion_vivienda: {situacion_vivienda_valores_fuera_rango}")
    
    indicador = (situacion_vivienda_valores_fuera_rango / cantidad_filas_creditos)
    print(f"Porcentaje de filas con errores de rango de valores (atributo situacion_vivienda): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_situacion_vivienda()

Cantidad de filas con valores fuera de rango en atributo situacion_vivienda: 0
Porcentaje de filas con errores de rango de valores (atributo situacion_vivienda): 0.0 %
Evaluación: ok


In [16]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: objetivo_credito

valores = df_creditos['objetivo_credito'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_creditos['objetivo_credito']) - df_creditos['objetivo_credito'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida
    
valores_validos = 'EDUCACIÓN|SALUD|INVERSIONES|PAGO_DEUDAS|PERSONAL|MEJORAS_HOGAR' 

# Se define una re de los valores validos según el anexo

df_creditos['objetivo_credito_ok'] = df_creditos['objetivo_credito'].astype(str).str.match(valores_validos)

print("Se visualizan las filas con errores de rango:")
display(df_creditos[~df_creditos['objetivo_credito_ok']]) # Para visualizar las tuplas con valores nulos o erróneos

# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_creditos[~df_creditos['objetivo_credito_ok']] 
print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
objetivo_credito
EDUCACIÓN        2328
SALUD            1853
INVERSIONES      1753
PAGO_DEUDAS      1673
PERSONAL         1643
MEJORAS_HOGAR     877
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok


Cantidad detectada: 0


In [21]:
# Verificación de valores para el atributo: pct_ingreso

# Mostrar distribución general
print(f"Distribución estadística del atributo 'pct_ingreso':\n{df_creditos['pct_ingreso'].describe()}\n")

# Contar valores nulos
cantidad_nulos = df_creditos['pct_ingreso'].isna().sum()
if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n")
else:
    print("No existen filas con valores nulos para este atributo.\n")

# Definir condiciones de valores válidos (pct_ingreso entre 0 y 1 incluido)
df_creditos['pct_ingreso_ok'] = df_creditos['pct_ingreso'].between(0, 1, inclusive='both')

# Mostrar filas que no cumplen la condición
errores = df_creditos[~df_creditos['pct_ingreso_ok']]
print("Se visualizan las filas con errores de rango:")
display(errores)

print(f"Cantidad detectada: {errores.shape[0]}")


def calcular_rangos_valores_pct_ingreso():
    pct_ingreso_valores_fuera_rango = errores.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo pct_ingreso: {pct_ingreso_valores_fuera_rango}")

    indicador = pct_ingreso_valores_fuera_rango / cantidad_filas_creditos
    print(f"Porcentaje de filas con errores de rango de valores (atributo pct_ingreso): {round(indicador * 100, 2)} %")

    if indicador > RANGOS_VALORES:
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

# Llamar a la función
calcular_rangos_valores_pct_ingreso()


Distribución estadística del atributo 'pct_ingreso':
count    10127.000000
mean         0.177226
std          0.109784
min          0.010000
25%          0.100000
50%          0.150000
75%          0.240000
max          0.830000
Name: pct_ingreso, dtype: float64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok,pct_ingreso_ok


Cantidad detectada: 0
Cantidad de filas con valores fuera de rango en atributo pct_ingreso: 0
Porcentaje de filas con errores de rango de valores (atributo pct_ingreso): 0.0 %
Evaluación: ok


In [22]:
# Verificación de valores para el atributo: tasa_interes

# Mostrar estadísticas básicas
print(f"Distribución estadística del atributo 'tasa_interes':\n{df_creditos['tasa_interes'].describe()}\n")

# Contar valores nulos
cantidad_nulos_tasa = df_creditos['tasa_interes'].isna().sum()
if cantidad_nulos_tasa > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos_tasa}\n")
else:
    print("No existen filas con valores nulos para este atributo.\n")

# Supongamos que tasa_interes debe estar entre 0% y 100% inclusive
df_creditos['tasa_interes_ok'] = df_creditos['tasa_interes'].between(0, 100, inclusive='both')

# Filtrar valores incorrectos
errores_tasa = df_creditos[~df_creditos['tasa_interes_ok']]
print("Se visualizan las filas con errores de rango:")
display(errores_tasa)

# Mostrar cantidad de errores detectados
print(f"Cantidad detectada: {errores_tasa.shape[0]}")

# Cantidad total de filas
cantidad_filas_creditos = len(df_creditos)

# Función de evaluación
def calcular_rangos_valores_tasa_interes():
    tasa_interes_valores_fuera_rango = errores_tasa.shape[0] + cantidad_nulos_tasa
    print(f"Cantidad de filas con valores fuera de rango en atributo tasa_interes: {tasa_interes_valores_fuera_rango}")
    
    indicador = tasa_interes_valores_fuera_rango / cantidad_filas_creditos
    print(f"Porcentaje de filas con errores de rango de valores (atributo tasa_interes): {round(indicador * 100, 2)} %")

    if indicador > RANGOS_VALORES:
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

# Llamada a la función
calcular_rangos_valores_tasa_interes()


Distribución estadística del atributo 'tasa_interes':
count    9215.000000
mean       10.979427
std         3.204760
min         5.420000
25%         7.900000
50%        10.990000
75%        13.430000
max        22.110000
Name: tasa_interes, dtype: float64

Cantidad de nulos en el atributo: 912

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok,pct_ingreso_ok,tasa_interes_ok
39,785432733.0,23,30000,4,3.0,ALQUILER,71500,PAGO_DEUDAS,0.42,NaN,1,N,0.083333,True,True,True,False
50,714374133.0,24,30000,4,4.0,ALQUILER,78000,PAGO_DEUDAS,0.38,NaN,1,Y,0.083333,True,True,True,False
57,716632758.0,23,35000,4,3.0,PROPIA,277000,PERSONAL,0.13,NaN,0,N,0.083333,True,True,True,False
59,712030833.0,24,1750,3,2.0,PROPIA,12000,INVERSIONES,0.15,NaN,0,Y,0.083333,True,True,True,False
62,711481533.0,26,10000,4,0.0,HIPOTECA,263000,EDUCACIÓN,0.04,NaN,1,N,0.083333,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10088,709188108.0,23,9500,3,1.0,ALQUILER,34000,INVERSIONES,0.28,NaN,1,N,0.083333,True,True,True,False
10090,712296408.0,22,12000,3,3.0,OTROS,59000,SALUD,0.20,NaN,1,Y,0.083333,True,True,True,False
10094,789023508.0,25,9500,3,1.0,ALQUILER,46000,SALUD,0.21,NaN,0,N,0.083333,True,True,True,False
10100,716644008.0,22,15000,2,1.0,OTROS,59000,PAGO_DEUDAS,0.25,NaN,1,Y,0.083333,True,True,True,False


Cantidad detectada: 912
Cantidad de filas con valores fuera de rango en atributo tasa_interes: 1824
Porcentaje de filas con errores de rango de valores (atributo tasa_interes): 18.01 %
Evaluación: no cumplimiento


In [23]:
# Verificación de valores para el atributo: estado_credito

# Mostrar distribución de valores
valores_estado = df_creditos['estado_credito'].value_counts()
print(f"Distribución inicial del atributo 'estado_credito':\n{valores_estado}\n")

# Contar valores nulos
cantidad_nulos_estado = df_creditos['estado_credito'].isna().sum()
if cantidad_nulos_estado > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos_estado}\n")
else:
    print("No existen filas con valores nulos para este atributo.\n")

# Definir valores válidos según el anexo
valores_validos_estado = '0|1'

# Validación de valores
# Se convierte a string y se evalúa con regex (puede haber valores tipo float o int)
df_creditos['estado_credito_ok'] = df_creditos['estado_credito'].astype(str).str.match(valores_validos_estado)

# Filtrar valores fuera de rango
errores_estado = df_creditos[~df_creditos['estado_credito_ok']]
print("Se visualizan las filas con errores de rango:")
display(errores_estado)

# Mostrar cantidad de errores detectados
print(f"Cantidad detectada: {errores_estado.shape[0]}")

# Cantidad total de filas
total_filas_creditos = len(df_creditos)

# Evaluación del atributo

def calcular_rangos_valores_estado_credito():
    estado_credito_valores_fuera_rango = errores_estado.shape[0] + cantidad_nulos_estado
    print(f"Cantidad de filas con valores fuera de rango en atributo estado_credito: {estado_credito_valores_fuera_rango}")

    indicador = estado_credito_valores_fuera_rango / total_filas_creditos
    print(f"Porcentaje de filas con errores de rango de valores (atributo estado_credito): {round(indicador * 100, 2)} %")

    if indicador > RANGOS_VALORES:
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

# Llamada a la función
calcular_rangos_valores_estado_credito()


Distribución inicial del atributo 'estado_credito':
estado_credito
0    7635
1    2492
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok,pct_ingreso_ok,tasa_interes_ok,estado_credito_ok


Cantidad detectada: 0
Cantidad de filas con valores fuera de rango en atributo estado_credito: 0
Porcentaje de filas con errores de rango de valores (atributo estado_credito): 0.0 %
Evaluación: ok


In [24]:
# Verificación de valores para el atributo: falta_pago

# Mostrar distribución de valores
valores_falta_pago = df_creditos['falta_pago'].value_counts()
print(f"Distribución inicial del atributo 'falta_pago':\n{valores_falta_pago}\n")

# Contar valores nulos
cantidad_nulos_falta_pago = df_creditos['falta_pago'].isna().sum()
if cantidad_nulos_falta_pago > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos_falta_pago}\n")
else:
    print("No existen filas con valores nulos para este atributo.\n")

# Definir valores válidos según el anexo
valores_validos_falta_pago = 'Y|N'

# Validación de valores
df_creditos['falta_pago_ok'] = df_creditos['falta_pago'].astype(str).str.match(valores_validos_falta_pago)

# Filtrar valores fuera de rango
errores_falta_pago = df_creditos[~df_creditos['falta_pago_ok']]
print("Se visualizan las filas con errores de rango:")
display(errores_falta_pago)

# Mostrar cantidad de errores detectados
print(f"Cantidad detectada: {errores_falta_pago.shape[0]}")

# Cantidad total de filas
total_filas_creditos = len(df_creditos)

# Evaluación del atributo
def calcular_rangos_valores_falta_pago():
    falta_pago_valores_fuera_rango = errores_falta_pago.shape[0] + cantidad_nulos_falta_pago
    print(f"Cantidad de filas con valores fuera de rango en atributo falta_pago: {falta_pago_valores_fuera_rango}")

    indicador = falta_pago_valores_fuera_rango / total_filas_creditos
    print(f"Porcentaje de filas con errores de rango de valores (atributo falta_pago): {round(indicador * 100, 2)} %")

    if indicador > RANGOS_VALORES:
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

# Llamada a la función
calcular_rangos_valores_falta_pago()


Distribución inicial del atributo 'falta_pago':
falta_pago
N    8359
Y    1768
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok,pct_ingreso_ok,tasa_interes_ok,estado_credito_ok,falta_pago_ok


Cantidad detectada: 0
Cantidad de filas con valores fuera de rango en atributo falta_pago: 0
Porcentaje de filas con errores de rango de valores (atributo falta_pago): 0.0 %
Evaluación: ok


----

Tarjetas

In [17]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: antiguedad_cliente

valores = df_tarjetas['antiguedad_cliente'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_tarjetas['antiguedad_cliente']) - df_tarjetas['antiguedad_cliente'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_tarjetas[df_tarjetas['antiguedad_cliente'] < 0]

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
antiguedad_cliente
36.0    2463
37.0     358
34.0     353
38.0     347
39.0     341
40.0     333
31.0     318
35.0     317
33.0     305
30.0     300
41.0     297
32.0     289
28.0     275
43.0     273
42.0     271
29.0     241
44.0     230
45.0     227
27.0     206
46.0     197
26.0     186
47.0     171
25.0     165
48.0     162
24.0     160
49.0     141
23.0     116
22.0     105
56.0     103
50.0      96
21.0      83
51.0      80
53.0      78
20.0      74
13.0      70
19.0      63
52.0      62
18.0      58
54.0      53
55.0      42
17.0      39
15.0      34
16.0      29
14.0      16
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,completitud_fila


Cantidad detectada: 0


In [18]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: personas_a_cargo

valores = df_tarjetas['personas_a_cargo'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_tarjetas['personas_a_cargo']) - df_tarjetas['personas_a_cargo'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_tarjetas[df_tarjetas['personas_a_cargo'] < 0]

print("Se visualizan las filas con errores de rango:")
display(resultado) # Para visualizar las tuplas con valores nulos o erróneos

print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
personas_a_cargo
3.0    2732
2.0    2655
1.0    1838
4.0    1574
0.0     904
5.0     424
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,completitud_fila


Cantidad detectada: 0


- nominales

In [19]:
#Verificar que los valores de cada atributo se encuentren dentro de los listados anexos

#Atributo: estado_civil

valores = df_tarjetas['estado_civil'].value_counts() #Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_tarjetas['estado_civil']) - df_tarjetas['estado_civil'].count() # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n") # Impresión de la cantidad de nulos
else:
    print("No existen filas con valores nulos para este atributo.\n")


# Se identifica y cuenta a los valores que no cumplen la condición definida
    
valores_validos = 'CASADO|SOLTERO|DESCONOCIDO|DIVORCIADO' # Se define una re de los valores validos según el anexo

df_tarjetas['estado_civil_ok'] = df_tarjetas['estado_civil'].astype(str).str.match(valores_validos)

print("Se visualizan las filas con errores de rango:")
display(df_tarjetas[~df_tarjetas['estado_civil_ok']]) # Para visualizar las tuplas con valores nulos o erróneos

# Se identifica y cuenta a los valores que no cumplen la condición definida

resultado = df_tarjetas[~df_tarjetas['estado_civil_ok']] 
print(f"Cantidad detectada: {resultado.shape[0]}")

Distribución inicial del atributo: 
estado_civil
CASADO         4687
SOLTERO        3943
DESCONOCIDO     749
DIVORCIADO      748
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,completitud_fila,estado_civil_ok


Cantidad detectada: 0


In [26]:
# Verificación de valores para el atributo: estado_cliente

# Mostrar distribución de valores
valores_estado_cliente = df_tarjetas['estado_cliente'].value_counts()
print(f"Distribución inicial del atributo 'estado_cliente':\n{valores_estado_cliente}\n")

# Contar valores nulos
cantidad_nulos_estado_cliente = df_tarjetas['estado_cliente'].isna().sum()
if cantidad_nulos_estado_cliente > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos_estado_cliente}\n")
else:
    print("No existen filas con valores nulos para este atributo.\n")

# Definir valores válidos según el anexo (presumiblemente: ACTIVO, INACTIVO, MOROSO, etc.)
valores_validos_estado_cliente = 'ACTIVO|PASIVO'

# Validación de valores
df_tarjetas['estado_cliente_ok'] = df_tarjetas['estado_cliente'].astype(str).str.match(valores_validos_estado_cliente)

# Filtrar valores fuera de rango
errores_estado_cliente = df_tarjetas[~df_tarjetas['estado_cliente_ok']]
print("Se visualizan las filas con errores de rango:")
display(errores_estado_cliente)

# Mostrar cantidad de errores detectados
print(f"Cantidad detectada: {errores_estado_cliente.shape[0]}")

# Cantidad total de filas
total_filas_tarjetas = len(df_tarjetas)

# Evaluación del atributo

def calcular_rangos_valores_estado_cliente():
    estado_cliente_valores_fuera_rango = errores_estado_cliente.shape[0] + cantidad_nulos_estado_cliente
    print(f"Cantidad de filas con valores fuera de rango en atributo estado_cliente: {estado_cliente_valores_fuera_rango}")

    indicador = estado_cliente_valores_fuera_rango / total_filas_tarjetas
    print(f"Porcentaje de filas con errores de rango de valores (atributo estado_cliente): {round(indicador * 100, 2)} %")

    if indicador > RANGOS_VALORES:
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

# Llamada a la función
calcular_rangos_valores_estado_cliente()


Distribución inicial del atributo 'estado_cliente':
estado_cliente
ACTIVO    8500
PASIVO    1627
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,completitud_fila,estado_civil_ok,estado_cliente_ok


Cantidad detectada: 0
Cantidad de filas con valores fuera de rango en atributo estado_cliente: 0
Porcentaje de filas con errores de rango de valores (atributo estado_cliente): 0.0 %
Evaluación: ok


In [27]:
# Validación del atributo: gastos_ult_12m

valores = df_tarjetas['gastos_ult_12m'].value_counts(dropna=False)
print(f"Distribución inicial del atributo:\n{valores}\n")

cantidad_nulos = df_tarjetas['gastos_ult_12m'].isna().sum()
if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n")
else:
    print("No existen filas con valores nulos para este atributo.\n")

# Se define la condición de valores aceptables: mayores o iguales a cero
condicion_valida = df_tarjetas['gastos_ult_12m'] >= 0

df_tarjetas['gastos_ult_12m_ok'] = condicion_valida

print("Se visualizan las filas con errores de rango:")
display(df_tarjetas[~df_tarjetas['gastos_ult_12m_ok']])

resultado = df_tarjetas[~df_tarjetas['gastos_ult_12m_ok']]
print(f"Cantidad detectada: {resultado.shape[0]}")

# Evaluación
cantidad_filas_tarjetas = df_tarjetas.shape[0]

def calcular_rangos_valores_gastos():
    gastos_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo gastos_ult_12m: {gastos_fuera_rango}")

    indicador = (gastos_fuera_rango / cantidad_filas_tarjetas)
    print(f"Porcentaje de filas con errores de rango de valores (atributo gastos_ult_12m): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_gastos()


Distribución inicial del atributo:
gastos_ult_12m
4253.0     11
4509.0     11
4518.0     10
2229.0     10
4220.0      9
           ..
10170.0     1
14638.0     1
16706.0     1
8531.0      1
14501.0     1
Name: count, Length: 5033, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,completitud_fila,estado_civil_ok,estado_cliente_ok,gastos_ult_12m_ok


Cantidad detectada: 0
Cantidad de filas con valores fuera de rango en atributo gastos_ult_12m: 0
Porcentaje de filas con errores de rango de valores (atributo gastos_ult_12m): 0.0 %
Evaluación: ok


In [28]:

# Conteo de ocurrencias por valor (not-null)
valores = df_tarjetas['genero'].value_counts()
print(f"Distribución inicial del atributo 'genero': \n{valores}\n")

# Conteo de nulos
cantidad_nulos = len(df_tarjetas['genero']) - df_tarjetas['genero'].count()
if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n")
else:
    print("No existen filas con valores nulos para este atributo.\n")

# Validación de valores correctos
valores_validos = 'M|F|DESCONOCIDO'
df_tarjetas['genero_ok'] = df_tarjetas['genero'].astype(str).str.match(valores_validos)

# Visualización de registros fuera de rango
errores = df_tarjetas[~df_tarjetas['genero_ok']]
print("Se visualizan las filas con errores de rango:")
display(errores)
print(f"Cantidad detectada: {errores.shape[0]}")

# Cálculo de porcentaje de errores y evaluación
def calcular_rangos_valores_genero():
    cantidad_filas_tarjetas = df_tarjetas.shape[0]
    genero_valores_fuera_rango = errores.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo genero: {genero_valores_fuera_rango}")

    indicador = genero_valores_fuera_rango / cantidad_filas_tarjetas
    print(f"Porcentaje de filas con errores de rango de valores (atributo genero): {round(indicador * 100, 2)} %")

    if indicador > RANGOS_VALORES:
        print("Evaluación: no cumplimiento")
    else:
        print("Evaluación: ok")

calcular_rangos_valores_genero()


Distribución inicial del atributo 'genero': 
genero
F    5358
M    4769
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,completitud_fila,estado_civil_ok,estado_cliente_ok,gastos_ult_12m_ok,genero_ok


Cantidad detectada: 0
Cantidad de filas con valores fuera de rango en atributo genero: 0
Porcentaje de filas con errores de rango de valores (atributo genero): 0.0 %
Evaluación: ok


In [29]:
# Validación del atributo: limite_credito_tc

# 1. Conteo de valores
valores = df_tarjetas['limite_credito_tc'].value_counts()  # Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo:\n{valores}\n")

# 2. Cantidad de nulos
cantidad_nulos = len(df_tarjetas['limite_credito_tc']) - df_tarjetas['limite_credito_tc'].count()
if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n")
else:
    print("No existen filas con valores nulos para este atributo.\n")

# 3. Validación de valores dentro de rango lógico (> 0)
df_tarjetas['limite_credito_tc_ok'] = df_tarjetas['limite_credito_tc'].apply(lambda x: x > 0)

print("Se visualizan las filas con errores de rango:")
display(df_tarjetas[~df_tarjetas['limite_credito_tc_ok']])

# 4. Conteo de valores fuera de rango
resultado = df_tarjetas[~df_tarjetas['limite_credito_tc_ok']]
print(f"Cantidad detectada: {resultado.shape[0]}")

# 5. Evaluación final del atributo

def calcular_rangos_valores_limite_credito():
    limite_credito_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo limite_credito_tc: {limite_credito_valores_fuera_rango}")

    cantidad_filas_tarjetas = df_tarjetas.shape[0]
    indicador = (limite_credito_valores_fuera_rango / cantidad_filas_tarjetas)
    print(f"Porcentaje de filas con errores de rango de valores (atributo limite_credito_tc): {round(indicador * 100, 2)} %")

    if indicador > RANGOS_VALORES:
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_limite_credito()

Distribución inicial del atributo:
limite_credito_tc
34516.0    508
1438.3     507
15987.0     18
9959.0      18
23981.0     12
          ... 
12540.0      1
7288.0       1
21906.0      1
4103.0       1
12958.0      1
Name: count, Length: 6205, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,completitud_fila,estado_civil_ok,estado_cliente_ok,gastos_ult_12m_ok,genero_ok,limite_credito_tc_ok


Cantidad detectada: 0
Cantidad de filas con valores fuera de rango en atributo limite_credito_tc: 0
Porcentaje de filas con errores de rango de valores (atributo limite_credito_tc): 0.0 %
Evaluación: ok


In [30]:
# Validación del atributo: nivel_educativo

# Conteo de ocurrencias por valor (not-null)
valores = df_tarjetas['nivel_educativo'].value_counts()
print(f"Distribución inicial del atributo 'nivel_educativo': \n{valores}\n")

# Conteo de valores nulos
cantidad_nulos = len(df_tarjetas['nivel_educativo']) - df_tarjetas['nivel_educativo'].count()
if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n")
else:
    print("No existen filas con valores nulos para este atributo.\n")

# Definición de valores válidos según el anexo
valores_validos = (
    "SECUNDARIO_COMPLETO|SECUNDARIO_INCOMPLETO|UNIVERSITARIO_COMPLETO|"
    "UNIVERSITARIO_INCOMPLETO|POSGRADO_COMPLETO|POSGRADO_INCOMPLETO|DESCONOCIDO"
)

# Evaluación del cumplimiento de los valores
valores_ok = df_tarjetas['nivel_educativo'].astype(str).str.match(valores_validos)
df_tarjetas['nivel_educativo_ok'] = valores_ok

# Visualización de filas inválidas
print("Se visualizan las filas con errores de rango:")
display(df_tarjetas[~df_tarjetas['nivel_educativo_ok']])

# Conteo de filas inválidas
errores = df_tarjetas[~df_tarjetas['nivel_educativo_ok']]
print(f"Cantidad detectada: {errores.shape[0]}")

# Evaluación general
def calcular_rangos_valores_nivel_educativo():
    filas_fuera_rango = errores.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en 'nivel_educativo': {filas_fuera_rango}")

    indicador = filas_fuera_rango / len(df_tarjetas)
    print(f"Porcentaje de filas con errores: {round(indicador * 100, 2)} %")

    if indicador > RANGOS_VALORES:
        print("Evaluación: no cumplimiento")
    else:
        print("Evaluación: ok")

calcular_rangos_valores_nivel_educativo()


Distribución inicial del atributo 'nivel_educativo': 
nivel_educativo
UNIVERSITARIO_COMPLETO      3128
UNIVERSITARIO_INCOMPLETO    2500
SECUNDARIO_COMPLETO         2013
DESCONOCIDO                 1519
POSGRADO_INCOMPLETO          516
POSGRADO_COMPLETO            451
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,completitud_fila,estado_civil_ok,estado_cliente_ok,gastos_ult_12m_ok,genero_ok,limite_credito_tc_ok,nivel_educativo_ok


Cantidad detectada: 0
Cantidad de filas con valores fuera de rango en 'nivel_educativo': 0
Porcentaje de filas con errores: 0.0 %
Evaluación: ok


In [31]:
# Verificación del atributo: nivel_tarjeta

valores = df_tarjetas['nivel_tarjeta'].value_counts()  # Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

cantidad_nulos = len(df_tarjetas['nivel_tarjeta']) - df_tarjetas['nivel_tarjeta'].count()  # Conteo de nulos

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n")
else:
    print("No existen filas con valores nulos para este atributo.\n")

# Se identifican los valores que no cumplen con los definidos como válidos
valores_validos = 'Blue|Silver|Gold|Platinum'

df_tarjetas['nivel_tarjeta_ok'] = df_tarjetas['nivel_tarjeta'].astype(str).str.match(valores_validos)

print("Se visualizan las filas con errores de rango:")
display(df_tarjetas[~df_tarjetas['nivel_tarjeta_ok']])

resultado = df_tarjetas[~df_tarjetas['nivel_tarjeta_ok']]
print(f"Cantidad detectada: {resultado.shape[0]}")

# Métrica final y evaluación

def calcular_rangos_valores_nivel_tarjeta():
    nivel_tarjeta_valores_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo nivel_tarjeta: {nivel_tarjeta_valores_fuera_rango}")

    cantidad_filas_tarjetas = len(df_tarjetas)
    indicador = (nivel_tarjeta_valores_fuera_rango / cantidad_filas_tarjetas)
    print(f"Porcentaje de filas con errores de rango de valores (atributo nivel_tarjeta): {round(indicador * 100, 2)} %")

    if (indicador > RANGOS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_nivel_tarjeta()


Distribución inicial del atributo: 
nivel_tarjeta
Blue        9436
Silver       555
Gold         116
Platinum      20
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,completitud_fila,estado_civil_ok,estado_cliente_ok,gastos_ult_12m_ok,genero_ok,limite_credito_tc_ok,nivel_educativo_ok,nivel_tarjeta_ok


Cantidad detectada: 0
Cantidad de filas con valores fuera de rango en atributo nivel_tarjeta: 0
Porcentaje de filas con errores de rango de valores (atributo nivel_tarjeta): 0.0 %
Evaluación: ok


In [33]:
import pandas as pd

# Suponiendo que ya tienes cargado el DataFrame df_tarjetas
valores = df_tarjetas['operaciones_ult_12m'].value_counts()
print(f"Distribución inicial del atributo:\n{valores}\n")

cantidad_nulos = len(df_tarjetas['operaciones_ult_12m']) - df_tarjetas['operaciones_ult_12m'].count()
if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n")
else:
    print("No existen filas con valores nulos para este atributo.\n")

# Definición de rango válido
minimo = 10
maximo = 139

# Se crea una columna auxiliar para validar el rango
df_tarjetas['operaciones_ult_12m_ok'] = df_tarjetas['operaciones_ult_12m'].apply(lambda x: minimo <= x <= maximo if pd.notnull(x) else False)

print("Se visualizan las filas con errores de rango:")
display(df_tarjetas[~df_tarjetas['operaciones_ult_12m_ok']])

# Cálculo de valores fuera de rango o nulos
resultado = df_tarjetas[~df_tarjetas['operaciones_ult_12m_ok']]
print(f"Cantidad detectada: {resultado.shape[0]}")

# Umbral de aceptación
cantidad_filas_tarjetas = df_tarjetas.shape[0]

def calcular_rangos_valores_operaciones_ult_12m():
    operaciones_fuera_rango = resultado.shape[0] + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo operaciones_ult_12m: {operaciones_fuera_rango}")

    indicador = (operaciones_fuera_rango / cantidad_filas_tarjetas)
    print(f"Porcentaje de filas con errores de rango de valores (atributo operaciones_ult_12m): {round(indicador * 100, 2)} %")

    if indicador > RANGOS_VALORES:
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_rangos_valores_operaciones_ult_12m()


Distribución inicial del atributo:
operaciones_ult_12m
81.0     208
75.0     203
71.0     203
69.0     202
82.0     202
        ... 
11.0       2
134.0      1
139.0      1
138.0      1
132.0      1
Name: count, Length: 126, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,completitud_fila,estado_civil_ok,estado_cliente_ok,gastos_ult_12m_ok,genero_ok,limite_credito_tc_ok,nivel_educativo_ok,nivel_tarjeta_ok,operaciones_ult_12m_ok


Cantidad detectada: 0
Cantidad de filas con valores fuera de rango en atributo operaciones_ult_12m: 0
Porcentaje de filas con errores de rango de valores (atributo operaciones_ult_12m): 0.0 %
Evaluación: ok


In [34]:
# Verificación del atributo: personas_a_cargo

# Mostrar la distribución inicial
valores = df_tarjetas['personas_a_cargo'].value_counts()  # Conteo de ocurrencias por valor (not-null)
print(f"Distribución inicial del atributo: \n{valores}\n")

# Conteo de nulos
cantidad_nulos = len(df_tarjetas['personas_a_cargo']) - df_tarjetas['personas_a_cargo'].count()

if cantidad_nulos > 0:
    print(f"Cantidad de nulos en el atributo: {cantidad_nulos}\n")
else:
    print("No existen filas con valores nulos para este atributo.\n")

# Verificación de valores dentro del rango permitido (por ejemplo, entre 0 y 10)
df_tarjetas['personas_a_cargo_ok'] = df_tarjetas['personas_a_cargo'].apply(lambda x: 0 <= x <= 10 if pd.notnull(x) else False)

# Visualizar filas con errores
errores = df_tarjetas[~df_tarjetas['personas_a_cargo_ok']]
print("Se visualizan las filas con errores de rango:")
display(errores)

# Cálculo de cantidad de errores y porcentaje respecto al total
cantidad_errores = errores.shape[0]
cantidad_total = df_tarjetas.shape[0]

print(f"Cantidad detectada: {cantidad_errores}")

def calcular_rangos_valores_personas_a_cargo():
    total_fuera_rango = cantidad_errores + cantidad_nulos
    print(f"Cantidad de filas con valores fuera de rango en atributo personas_a_cargo: {total_fuera_rango}")

    indicador = total_fuera_rango / cantidad_total
    print(f"Porcentaje de filas con errores de rango de valores (atributo personas_a_cargo): {round(indicador * 100, 2)} %")

    if indicador > RANGOS_VALORES:
        print("Evaluación: no cumplimiento")
    else:
        print("Evaluación: ok")

calcular_rangos_valores_personas_a_cargo()


Distribución inicial del atributo: 
personas_a_cargo
3.0    2732
2.0    2655
1.0    1838
4.0    1574
0.0     904
5.0     424
Name: count, dtype: int64

No existen filas con valores nulos para este atributo.

Se visualizan las filas con errores de rango:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,...,completitud_fila,estado_civil_ok,estado_cliente_ok,gastos_ult_12m_ok,genero_ok,limite_credito_tc_ok,nivel_educativo_ok,nivel_tarjeta_ok,operaciones_ult_12m_ok,personas_a_cargo_ok


Cantidad detectada: 0
Cantidad de filas con valores fuera de rango en atributo personas_a_cargo: 0
Porcentaje de filas con errores de rango de valores (atributo personas_a_cargo): 0.0 %
Evaluación: ok


---

## Dimensión: consistencia

### (4) Claves únicas

Dataset: **datos_creditos**

In [35]:
#Se obtiene el valor de la cantidad de filas actual
cant_antes = df_creditos.shape[0] 

# Se ordena el dataset según el atributo que se desee evaluar (requerido para el paso siguiente)
df_creditos.sort_values("id_cliente", inplace=True)

# Se detectan y eliminan los duplicados en un atributo dejando la última ocurrencia
df_creditos.drop_duplicates(subset ="id_cliente", keep = 'last', inplace = True)

# Se obtiene el valor posterior a la operación
cant_despues = df_creditos.shape[0]

# Se imprimen ambos valores
print('Dataset: creditos')
print(f"Antes del análisis de duplicados: {cant_antes} - Despues del filtrado de duplicados: {cant_despues}")
if cant_antes > cant_despues:
    diferencia = cant_antes - cant_despues
    pct_diferencia = ((cant_antes - cant_despues) / cant_antes) * 100
    print(f"Se detectaron claves duplicadas en {diferencia} fila(s) un {round(pct_diferencia, 2)}%.")
else:
    print("No se detectaron claves duplicadas")

Dataset: creditos
Antes del análisis de duplicados: 10127 - Despues del filtrado de duplicados: 10127
No se detectaron claves duplicadas


Dataset: **datos_tarjetas**

In [36]:
# Se obtiene el valor de la cantidad de filas actual
cant_antes = df_tarjetas.shape[0]  

# Se ordena el dataset según el atributo que se desee evaluar (requerido para el paso siguiente)
df_tarjetas.sort_values("id_cliente", inplace=True)

# Se detectan y eliminan los duplicados en un atributo dejando la última ocurrencia
df_tarjetas.drop_duplicates(subset ="id_cliente", keep = 'last', inplace = True)

# Se obtiene el valor posterior a la operación
cant_despues = df_tarjetas.shape[0]

#Se imprimen ambos valores
print('Dataset: tarjetas')
print(f"Antes del análisis de duplicados: {cant_antes} - Despues del filtrado de duplicados: {cant_despues}")
if cant_antes > cant_despues:
    diferencia = cant_antes - cant_despues
    pct_diferencia = ((cant_antes - cant_despues) / cant_antes) * 100
    print(f"Se detectaron claves duplicadas en {diferencia} fila(s) un {round(pct_diferencia, 2)}%.")
else:
    print("No se detectaron claves duplicadas")

Dataset: tarjetas
Antes del análisis de duplicados: 10127 - Despues del filtrado de duplicados: 10127
No se detectaron claves duplicadas


### (5) Integridad referencial

In [37]:
# Las uniones se hacen de a pares - revisar nombres de atributos

df_integrado = pd.merge(df_creditos, df_tarjetas, on='id_cliente', how='inner')
coincidencias = df_integrado.shape[0]

print(f"Datos de créditos: {cantidad_filas_creditos} - Coincidencias con datos de tarjetas: {coincidencias}")

print("\nSe visualiza el dataset resultante:")
display(df_integrado.head(5))

print(f"Reporte general:\n \
- Filas del dataset creditos (inicial): {cantidad_filas_creditos}\n \
- Filas del dataset tarjetas (inicial): {cantidad_filas_tarjetas}\n \
- Errores detectados en la operación de unión: {abs(coincidencias - cantidad_filas_creditos)} \n \
- Filas del dataset unificado: {df_integrado.shape[0]}")

Datos de créditos: 10127 - Coincidencias con datos de tarjetas: 10127

Se visualiza el dataset resultante:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,completitud_fila_y,estado_civil_ok,estado_cliente_ok,gastos_ult_12m_ok,genero_ok,limite_credito_tc_ok,nivel_educativo_ok,nivel_tarjeta_ok,operaciones_ult_12m_ok,personas_a_cargo_ok
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,...,0.0,True,True,True,True,True,True,True,True,True
1,708083283.0,24,1500,2,0.0,ALQUILER,30996,MEJORAS_HOGAR,0.05,15.99,...,0.0,True,True,True,True,True,True,True,True,True
2,708084558.0,23,10000,2,7.0,OTROS,40104,EDUCACIÓN,0.25,12.72,...,0.0,True,True,True,True,True,True,True,True,True
3,708085458.0,25,6000,4,2.0,ALQUILER,23198,INVERSIONES,0.26,8.00,...,0.0,True,True,True,True,True,True,True,True,True
4,708086958.0,26,10000,2,0.0,HIPOTECA,50000,EDUCACIÓN,0.20,7.74,...,0.0,True,True,True,True,True,True,True,True,True


Reporte general:
 - Filas del dataset creditos (inicial): 10127
 - Filas del dataset tarjetas (inicial): 10127
 - Errores detectados en la operación de unión: 0 
 - Filas del dataset unificado: 10127


In [ ]:
def calcular_integridad_referencial():
    cant_problemas = cantidad_filas_creditos - df_integrado.shape[0] # Se calcula sobre el inicio (foco)
    print(f"Casos de problemas de integridad referencial: {cant_problemas}")

    indicador = (cant_problemas / cantidad_filas_creditos)
    print(f"Porcentaje de filas con problemas de integridad referencial: {round(indicador * 100, 2)} %")

    if (indicador > INTEGRIDAD_REF):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')
        
calcular_integridad_referencial()

Casos de problemas de integridad referencial: 0
Porcentaje de filas con problemas de integridad referencial: 0.0 %
Evaluación: ok


---

## Dimensión exactitud (bis)
### (6) Reglas en valores

Regla 1: Para aquellos casos en que los créditos constituyan un porcentaje de los ingresos del cliente mayor al 50% sus ingresos deberán ser mayores a 20.000.

Regla 2: Para aquellos créditos cuya duración sea la mínima permitida el porcentaje de los ingresos del cliente (con respecto al importe solicitado) no podrá exceder el 60% salvo en los casos en los que sea propietario de su vivienda.

In [ ]:
#Se puede definir una función para aplicar los cálculos
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    ingresos = row.ingresos
    
    if pct_ingreso > 0.5 and ingresos <= 20000:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'


# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_integrado.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_resultado = pd.concat([df_integrado, regla_pct_ingresos], axis=1)  

# Se visualizan los datos
print("Se visualizan las tuplas que no cumplen con la regla:\n")
display(df_resultado[df_resultado.regla_pct_ingresos == 'err'].head())


# Se verifica la cantidad de elementos
aux = df_resultado[df_resultado.regla_pct_ingresos == 'err']
print(f"Cantidad de filas que no cumplen la regla: {aux.shape[0]}")

Se visualizan las tuplas que no cumplen con la regla:



,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,estado_civil_ok,estado_cliente_ok,gastos_ult_12m_ok,genero_ok,limite_credito_tc_ok,nivel_educativo_ok,nivel_tarjeta_ok,operaciones_ult_12m_ok,personas_a_cargo_ok,regla_pct_ingresos
466,709040508.0,21,15000,2,0.0,HIPOTECA,19500,EDUCACIÓN,0.77,9.64,...,True,True,True,True,True,True,True,True,True,err
1550,711232608.0,21,8000,4,3.0,ALQUILER,15500,MEJORAS_HOGAR,0.52,7.51,...,True,True,True,True,True,True,True,True,True,err
1900,711864783.0,24,10000,2,0.0,HIPOTECA,18000,PAGO_DEUDAS,0.56,12.18,...,True,True,True,True,True,True,True,True,True,err
2573,713115483.0,22,7000,2,0.0,ALQUILER,10000,MEJORAS_HOGAR,0.70,12.21,...,True,True,True,True,True,True,True,True,True,err
2809,713555133.0,22,8000,4,3.0,ALQUILER,14400,MEJORAS_HOGAR,0.56,9.99,...,True,True,True,True,True,True,True,True,True,err


Cantidad de filas que no cumplen la regla: 15


In [40]:
def calcular_regla_1():
    cant_problemas = aux.shape[0] # Se calcula sobre el inicio (foco)
    print(f"Casos de problemas de no cumplimiento de la regla 1: {cant_problemas}")

    indicador = (cant_problemas / df_integrado.shape[0])
    print(f"Porcentaje de filas con problemas de integridad referencial: {round(indicador * 100, 2)} %")

    if (indicador > REGLAS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

calcular_regla_1()

Casos de problemas de no cumplimiento de la regla 1: 15
Porcentaje de filas con problemas de integridad referencial: 0.15 %
Evaluación: ok


In [41]:
# Definimos la función para la Regla 2
def regla_duracion_pct_ingresos(row):
    duracion_credito = row.duracion_credito
    pct_ingreso = row.pct_ingreso
    situacion_vivienda = row.situacion_vivienda

    # Validamos si la duración es la mínima (asumimos que es 2, según el dataset compartido)
    if duracion_credito == 2 and pct_ingreso > 0.6 and situacion_vivienda != 'PROPIA':
        return 'err'  # No cumple la regla
    else:
        return 'ok'  # Cumple la regla


# Aplicamos la regla 2 al DataFrame
df_resultado["regla_duracion_pct_ingresos"] = df_resultado.apply(lambda row: regla_duracion_pct_ingresos(row), axis=1)

# Mostramos las filas que no cumplen la regla
print("Se visualizan las tuplas que no cumplen con la regla 2:\n")
display(df_resultado[df_resultado.regla_duracion_pct_ingresos == 'err'].head())

# Verificamos la cantidad de errores
aux = df_resultado[df_resultado.regla_duracion_pct_ingresos == 'err']
print(f"Cantidad de filas que no cumplen la regla 2: {aux.shape[0]}")

Se visualizan las tuplas que no cumplen con la regla 2:



,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,estado_cliente_ok,gastos_ult_12m_ok,genero_ok,limite_credito_tc_ok,nivel_educativo_ok,nivel_tarjeta_ok,operaciones_ult_12m_ok,personas_a_cargo_ok,regla_pct_ingresos,regla_duracion_pct_ingresos
466,709040508.0,21,15000,2,0.0,HIPOTECA,19500,EDUCACIÓN,0.77,9.64,...,True,True,True,True,True,True,True,True,err,err
541,709186983.0,23,20000,2,1.0,ALQUILER,32900,EDUCACIÓN,0.61,16.00,...,True,True,True,True,True,True,True,True,ok,err
1646,711399408.0,24,29100,2,8.0,HIPOTECA,46000,PERSONAL,0.63,12.99,...,True,True,True,True,True,True,True,True,ok,err
2573,713115483.0,22,7000,2,0.0,ALQUILER,10000,MEJORAS_HOGAR,0.70,12.21,...,True,True,True,True,True,True,True,True,err,err
3041,713962233.0,22,30000,2,1.0,ALQUILER,48000,EDUCACIÓN,0.63,18.39,...,True,True,True,True,True,True,True,True,ok,err


Cantidad de filas que no cumplen la regla 2: 7


In [ ]:
# Calculadora para Regla 2 (con umbral)
def calcular_regla_2():
    cant_problemas = aux.shape[0]  
    print(f"Casos de problemas de no cumplimiento de la regla 2: {cant_problemas}")

    indicador = (cant_problemas / df_integrado.shape[0])
    print(f"Porcentaje de filas con problemas de integridad referencial (regla 2): {round(indicador * 100, 2)} %")

    if (indicador > REGLAS_VALORES):
        print('Evaluación: no cumplimiento')
    else:
        print('Evaluación: ok')

# Llamado a la función
df_integrado = df_resultado  
calcular_regla_2()

Casos de problemas de no cumplimiento de la regla 2: 7
Porcentaje de filas con problemas de integridad referencial (regla 2): 0.07 %
Evaluación: ok


Exportación del dataset integrado

In [43]:
df_integrado.to_csv("../../data/processed/datos_integrados_c.csv", sep=";", index=False)